The dataset includes several data wranglings. The most important issue is that the columns Xi which indicate the day should be rows instead of columns. Also the measurements should be columns instead of rows.

Also we can observe that there are some null values and some inconsistencies with some of the rows. 

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('weather.csv', index_col=0)

In [4]:
df.head()

,X,year,month,measure,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31
0,1,2014,12,Max.TemperatureF,64,42,51,43,42,45,38,29,49,48,39,39,42,45,42,44,49,44,37,36,36,44,47,46,59,50,52,52,41,30,30
1,2,2014,12,Mean.TemperatureF,52,38,44,37,34,42,30,24,39,43,36,35,37,39,37,40,45,40,33,32,33,39,45,44,52,44,45,46,36,26,25
2,3,2014,12,Min.TemperatureF,39,33,37,30,26,38,21,18,29,38,32,31,32,33,32,35,41,36,29,27,30,33,42,41,44,37,38,40,30,22,20
3,4,2014,12,Max.Dew.PointF,46,40,49,24,37,45,36,28,49,45,37,28,28,29,33,42,46,34,25,30,30,39,45,46,58,31,34,42,26,10,8
4,5,2014,12,MeanDew.PointF,40,27,42,21,25,40,20,16,41,39,31,27,26,27,29,36,41,30,22,24,27,34,42,44,43,29,31,35,20,4,5


Column X is just an index, so let's drop it!

In [5]:
df.drop(['X'], axis=1, inplace=True)

In [114]:
df2 = df.pivot(index=['year', 'month'], columns='measure').stack(0).reset_index()
df2.rename(columns={'level_2': "day"}, inplace=True)
df2.columns.name = None

In [80]:
df2

,year,month,day,CloudCover,Events,Max.Dew.PointF,Max.Gust.SpeedMPH,Max.Humidity,Max.Sea.Level.PressureIn,Max.TemperatureF,Max.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Humidity,Mean.Sea.Level.PressureIn,Mean.TemperatureF,Mean.VisibilityMiles,Mean.Wind.SpeedMPH,MeanDew.PointF,Min.DewpointF,Min.Humidity,Min.Sea.Level.PressureIn,Min.TemperatureF,Min.VisibilityMiles,PrecipitationIn,WindDirDegrees
0,2014,12,X1,6,Rain,46,29,74,30.45,64,10,22,63,30.13,52,10,13,40,26,52,30.01,39,10,0.01,268
1,2014,12,X10,8,Rain,45,29,100,29.58,48,10,23,95,29.5,43,3,13,39,37,89,29.43,38,1,0.28,357
2,2014,12,X11,8,Rain-Snow,37,28,92,29.81,39,10,21,87,29.61,36,7,13,31,27,82,29.44,32,1,0.02,230
3,2014,12,X12,7,Snow,28,21,85,29.88,39,10,16,75,29.85,35,10,11,27,25,64,29.81,31,7,T,286
4,2014,12,X13,5,NaN,28,23,75,29.86,42,10,17,65,29.82,37,10,12,26,24,55,29.78,32,10,T,298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,2015,11,X6,4,NaN,62,32,93,30.07,73,10,26,79,29.9,68,10,15,61,54,64,29.71,62,10,0.00,222
362,2015,11,X7,6,NaN,45,33,57,30.02,69,10,25,48,29.93,60,10,13,38,33,39,29.83,50,10,0.00,280
363,2015,11,X8,0,NaN,34,25,65,30.38,56,10,18,48,30.25,50,10,12,30,24,30,30.04,44,10,0.00,283
364,2015,11,X9,2,NaN,36,20,70,30.43,60,10,16,52,30.37,51,10,9,32,30,33,30.32,41,10,0.00,237


In [115]:
dates = pd.to_datetime(df2.apply(lambda x: f"{x[0]}-{x[1]}-{x[2][1:]}", axis=1))
df2.insert(0, 'Date', dates)
df2.drop(columns=['year', 'month', 'day'], inplace=True)
df2.head()

,Date,CloudCover,Events,Max.Dew.PointF,Max.Gust.SpeedMPH,Max.Humidity,Max.Sea.Level.PressureIn,Max.TemperatureF,Max.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Humidity,Mean.Sea.Level.PressureIn,Mean.TemperatureF,Mean.VisibilityMiles,Mean.Wind.SpeedMPH,MeanDew.PointF,Min.DewpointF,Min.Humidity,Min.Sea.Level.PressureIn,Min.TemperatureF,Min.VisibilityMiles,PrecipitationIn,WindDirDegrees
0,2014-12-01,6,Rain,46,29,74,30.45,64,10,22,63,30.13,52,10,13,40,26,52,30.01,39,10,0.01,268
1,2014-12-10,8,Rain,45,29,100,29.58,48,10,23,95,29.5,43,3,13,39,37,89,29.43,38,1,0.28,357
2,2014-12-11,8,Rain-Snow,37,28,92,29.81,39,10,21,87,29.61,36,7,13,31,27,82,29.44,32,1,0.02,230
3,2014-12-12,7,Snow,28,21,85,29.88,39,10,16,75,29.85,35,10,11,27,25,64,29.81,31,7,T,286
4,2014-12-13,5,NaN,28,23,75,29.86,42,10,17,65,29.82,37,10,12,26,24,55,29.78,32,10,T,298


We can see in the above cell that some values in the `PrecipitationIn` column are set to *T* which probably mean NaN. We can subsitute them with 0 for that column:

In [116]:
df2['PrecipitationIn'].replace(['T','0'], inplace=True)

Let's go for columns with empty values:

In [118]:
df2.isna().any()

Date                         False
CloudCover                   False
Events                        True
Max.Dew.PointF               False
Max.Gust.SpeedMPH             True
Max.Humidity                 False
Max.Sea.Level.PressureIn     False
Max.TemperatureF             False
Max.VisibilityMiles          False
Max.Wind.SpeedMPH            False
Mean.Humidity                False
Mean.Sea.Level.PressureIn    False
Mean.TemperatureF            False
Mean.VisibilityMiles         False
Mean.Wind.SpeedMPH           False
MeanDew.PointF               False
Min.DewpointF                False
Min.Humidity                 False
Min.Sea.Level.PressureIn     False
Min.TemperatureF             False
Min.VisibilityMiles          False
PrecipitationIn              False
WindDirDegrees               False
dtype: bool

Let's fill in the empty values:

In [121]:
df2['Max.Gust.SpeedMPH'].unique()

array(['29', '28', '21', '23', '20', '10', '26', '30', '22', '25', '15',
       '40', '31', '38', '33', '32', '52', '24', '16', '39', '17', '43',
       '45', '44', '48', '49', '35', '37', '51', '41', '18', '94', '36',
       nan, '0', '12', '14'], dtype=object)

In [123]:
df2.fillna({'Events': 'Sunny', 'Max.Gust.SpeedMPH': '0'}, inplace=True)

In [127]:
df2.drop(columns=['Date', 'Events']).describe()

,CloudCover,Max.Dew.PointF,Max.Gust.SpeedMPH,Max.Humidity,Max.Sea.Level.PressureIn,Max.TemperatureF,Max.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Humidity,Mean.Sea.Level.PressureIn,Mean.TemperatureF,Mean.VisibilityMiles,Mean.Wind.SpeedMPH,MeanDew.PointF,Min.DewpointF,Min.Humidity,Min.Sea.Level.PressureIn,Min.TemperatureF,Min.VisibilityMiles,PrecipitationIn,WindDirDegrees
count,366,366,366,366,366,366,366,366,366,366,366,366,366,366,366,366,366,366,366,366,366
unique,9,73,36,51,99,76,5,26,66,100,74,11,19,80,83,74,102,75,11,58,220
top,6,61,28,100,30.22,82,10,20,66,30.13,75,10,9,59,57,39,30.09,64,10,0.00,272
freq,57,16,25,73,12,11,359,32,13,13,12,243,58,12,11,16,14,13,198,217,6


In [130]:
numerical_cols = df2.columns.difference(['Date', 'Events'])

In [132]:
df2[numerical_cols] = df2[numerical_cols].apply(pd.to_numeric, errors='ignore')

In [135]:
df2.describe()

,CloudCover,Max.Dew.PointF,Max.Gust.SpeedMPH,Max.Humidity,Max.Sea.Level.PressureIn,Max.TemperatureF,Max.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Humidity,Mean.Sea.Level.PressureIn,Mean.TemperatureF,Mean.VisibilityMiles,Mean.Wind.SpeedMPH,MeanDew.PointF,Min.DewpointF,Min.Humidity,Min.Sea.Level.PressureIn,Min.TemperatureF,Min.VisibilityMiles,PrecipitationIn,WindDirDegrees
count,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000
mean,4.707650,45.475410,26.546448,85.685792,30.155328,58.931694,9.907104,20.620219,66.021858,30.038251,51.404372,8.860656,10.680328,38.959016,32.245902,48.308743,29.925956,43.327869,6.715847,0.120546,200.081967
std,2.373206,18.317241,9.226631,49.996900,0.232095,20.047156,0.707768,5.762945,14.433392,0.235583,19.156754,2.065963,3.491515,19.870153,21.811639,17.414221,0.241768,18.734530,4.051196,0.327986,98.707393
min,0.000000,-6.000000,0.000000,39.000000,29.580000,18.000000,2.000000,8.000000,28.000000,29.490000,8.000000,-1.000000,4.000000,-11.000000,-18.000000,16.000000,29.160000,-3.000000,0.000000,0.000000,1.000000
25%,3.000000,32.000000,21.000000,73.250000,30.000000,42.000000,10.000000,16.000000,56.000000,29.870000,36.250000,8.000000,8.000000,24.000000,16.250000,35.000000,29.760000,30.000000,2.000000,0.000000,113.000000
50%,5.000000,47.500000,25.000000,86.000000,30.140000,60.000000,10.000000,20.000000,66.000000,30.030000,53.500000,10.000000,10.000000,41.000000,35.000000,46.000000,29.940000,46.000000,10.000000,0.000000,222.000000
75%,7.000000,61.000000,31.000000,93.000000,30.310000,76.000000,10.000000,24.000000,76.750000,30.190000,68.000000,10.000000,13.000000,56.000000,51.000000,60.000000,30.090000,60.000000,10.000000,0.070000,275.000000
max,8.000000,75.000000,94.000000,1000.000000,30.880000,96.000000,10.000000,38.000000,98.000000,30.770000,84.000000,10.000000,22.000000,71.000000,68.000000,96.000000,30.640000,74.000000,10.000000,2.900000,360.000000


As we can see in the cell above, `Mean.VisibilityMiles` has some rows with value -1 which is not a valid mean value. So let's change that:

In [137]:
df2.loc[df2['Mean.VisibilityMiles'] < 0, 'Mean.VisibilityMiles'] = 0

We can also observe that in some rows `Max.Humidity` is above 100% so let's fix that too:

In [138]:
df2.loc[df2['Max.Humidity'] > 100, 'Max.Humidity'] = 100

Let's check the statistics again:

In [139]:
df2.describe()

,CloudCover,Max.Dew.PointF,Max.Gust.SpeedMPH,Max.Humidity,Max.Sea.Level.PressureIn,Max.TemperatureF,Max.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Humidity,Mean.Sea.Level.PressureIn,Mean.TemperatureF,Mean.VisibilityMiles,Mean.Wind.SpeedMPH,MeanDew.PointF,Min.DewpointF,Min.Humidity,Min.Sea.Level.PressureIn,Min.TemperatureF,Min.VisibilityMiles,PrecipitationIn,WindDirDegrees
count,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000
mean,4.707650,45.475410,26.546448,83.226776,30.155328,58.931694,9.907104,20.620219,66.021858,30.038251,51.404372,8.863388,10.680328,38.959016,32.245902,48.308743,29.925956,43.327869,6.715847,0.120546,200.081967
std,2.373206,18.317241,9.226631,14.277886,0.232095,20.047156,0.707768,5.762945,14.433392,0.235583,19.156754,2.053510,3.491515,19.870153,21.811639,17.414221,0.241768,18.734530,4.051196,0.327986,98.707393
min,0.000000,-6.000000,0.000000,39.000000,29.580000,18.000000,2.000000,8.000000,28.000000,29.490000,8.000000,0.000000,4.000000,-11.000000,-18.000000,16.000000,29.160000,-3.000000,0.000000,0.000000,1.000000
25%,3.000000,32.000000,21.000000,73.250000,30.000000,42.000000,10.000000,16.000000,56.000000,29.870000,36.250000,8.000000,8.000000,24.000000,16.250000,35.000000,29.760000,30.000000,2.000000,0.000000,113.000000
50%,5.000000,47.500000,25.000000,86.000000,30.140000,60.000000,10.000000,20.000000,66.000000,30.030000,53.500000,10.000000,10.000000,41.000000,35.000000,46.000000,29.940000,46.000000,10.000000,0.000000,222.000000
75%,7.000000,61.000000,31.000000,93.000000,30.310000,76.000000,10.000000,24.000000,76.750000,30.190000,68.000000,10.000000,13.000000,56.000000,51.000000,60.000000,30.090000,60.000000,10.000000,0.070000,275.000000
max,8.000000,75.000000,94.000000,100.000000,30.880000,96.000000,10.000000,38.000000,98.000000,30.770000,84.000000,10.000000,22.000000,71.000000,68.000000,96.000000,30.640000,74.000000,10.000000,2.900000,360.000000
